# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,ilulissat,69.2167,-51.1000,-4.99,63,75,6.69,GL,1700668960
1,1,yermentau,51.6214,73.1069,2.46,86,100,3.74,KZ,1700668952
2,2,wailua homesteads,22.0669,-159.3780,22.08,89,63,1.79,US,1700668960
3,3,susuman,62.7833,148.1667,-21.29,98,100,2.51,RU,1700668960
4,4,klyuchi,52.2667,79.1667,-3.40,50,100,5.35,RU,1700668960


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.6,
    color = "City"
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather = city_data_df[(city_data_df["Cloudiness"] < 10) & (city_data_df["Wind Speed"] < 4) &
                             (city_data_df["Max Temp"] > 21) & (city_data_df["Max Temp"] < 30)]

# Drop any rows with null values
ideal_weather = ideal_weather.dropna(how = 'any')

# Display sample data
ideal_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,cabo san lucas,22.8909,-109.9124,26.40,50,0,0.89,MX,1700668840
39,39,darwin,-12.4611,130.8418,25.99,89,8,0.00,AU,1700668964
40,40,howrah,22.5892,88.3103,23.00,64,0,0.00,IN,1700668964
85,85,port elizabeth,-33.9180,25.5701,22.27,66,3,2.04,ZA,1700668889
98,98,paradip garh,20.3167,86.6167,24.17,63,0,3.21,IN,1700668971
138,138,port saint john's,-31.6229,29.5448,24.70,81,0,2.93,ZA,1700668975
154,154,sangod,24.9167,76.2833,21.03,32,1,1.42,IN,1700668977
163,163,ratnagiri,16.9833,73.3000,26.37,66,5,3.98,IN,1700668978
165,165,vallenar,-28.5708,-70.7581,27.58,25,0,3.81,CL,1700668687
191,191,kruisfontein,-34.0033,24.7314,23.48,36,0,3.73,ZA,1700668982


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather[["City", "Country", "Lat", 'Lng', "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,cabo san lucas,MX,22.8909,-109.9124,50,
39,darwin,AU,-12.4611,130.8418,89,
40,howrah,IN,22.5892,88.3103,64,
85,port elizabeth,ZA,-33.9180,25.5701,66,
98,paradip garh,IN,20.3167,86.6167,63,
138,port saint john's,ZA,-31.6229,29.5448,81,
154,sangod,IN,24.9167,76.2833,32,
163,ratnagiri,IN,16.9833,73.3000,66,
165,vallenar,CL,-28.5708,-70.7581,25,
191,kruisfontein,ZA,-34.0033,24.7314,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [9]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":  "accommodation.hotel",
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
cabo san lucas - nearest hotel: Comfort Rooms
darwin - nearest hotel: Mantra Pantanas Darwin
howrah - nearest hotel: Sun India Guest House
port elizabeth - nearest hotel: Waterford Hotel
paradip garh - nearest hotel: Hotel Sea Pearl
port saint john's - nearest hotel: Outback Inn
sangod - nearest hotel: No hotel found
ratnagiri - nearest hotel: Vihar
vallenar - nearest hotel: Hotel del Marqués
kruisfontein - nearest hotel: No hotel found
dolores - nearest hotel: Plaza Hotel
sur - nearest hotel: Sur Hotel
kailua-kona - nearest hotel: Kona Seaside Hotel
konarka - nearest hotel: Hotel Sun Village
san luis de la loma - nearest hotel: No hotel found
mahasamund - nearest hotel: No hotel found
diu - nearest hotel: Hotel Ankur
sabha - nearest hotel: فندق الكاظم-سبها
howard springs - nearest hotel: No hotel found
mawlamyinegyunn - nearest hotel: No hotel found
srivardhan - nearest hotel: No hotel found
richards bay - nearest hotel: SUN1 Richards Bay


,City,Country,Lat,Lng,Humidity,Hotel Name
28,cabo san lucas,MX,22.8909,-109.9124,50,Comfort Rooms
39,darwin,AU,-12.4611,130.8418,89,Mantra Pantanas Darwin
40,howrah,IN,22.5892,88.3103,64,Sun India Guest House
85,port elizabeth,ZA,-33.9180,25.5701,66,Waterford Hotel
98,paradip garh,IN,20.3167,86.6167,63,Hotel Sea Pearl
138,port saint john's,ZA,-31.6229,29.5448,81,Outback Inn
154,sangod,IN,24.9167,76.2833,32,No hotel found
163,ratnagiri,IN,16.9833,73.3000,66,Vihar
165,vallenar,CL,-28.5708,-70.7581,25,Hotel del Marqués
191,kruisfontein,ZA,-34.0033,24.7314,36,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [10]:
%%capture --no-display

# Configure the map plot
map_hotel_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols= ["Country","Hotel Name"])

# Display the map
map_hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)